In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [6]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [8]:
df = pd.read_csv('./ready_to_go2.csv')

In [10]:
# Aplicar la transformación logarítmica al precio
df['log_price'] = np.log(df['price'])

In [12]:
df.columns

Index(['Unnamed: 0', 'characteristics_bedrooms', 'characteristics_bathrooms',
       'characteristics_garages', 'characteristics_area', 'location_lat',
       'location_lng', 'price', 'neighborhood_name', 'district_name',
       'extras_Terreno', 'price_per_m2', 'built_area_ratio',
       'district_avg_price', 'neighborhood_avg_price', 'log_price'],
      dtype='object')

In [14]:
# Seleccionar las columnas para el modelo
features = ['characteristics_bedrooms', 'characteristics_bathrooms', 'characteristics_garages',
            'characteristics_area', 'location_lat', 'location_lng', 'extras_Terreno', 'price_per_m2']

In [16]:
# Definir X (características) e y (precio logarítmico)
X = df[features]
y = df['log_price']

In [18]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Crear el modelo XGBoost
xgb_model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}


In [22]:
# Definir otro espacio de búsqueda de hiperparámetros
param_grid_alt = {
    'n_estimators': [50, 150, 250],  # Probar con menos árboles para ver si el modelo necesita menos ajustes
    'max_depth': [4, 6, 8],  # Explorar profundidades más grandes para capturar interacciones más complejas
    'learning_rate': [0.05, 0.15, 0.25],  # Variar ligeramente la tasa de aprendizaje
    'subsample': [0.7, 0.85, 1.0],  # Reducir el tamaño de las muestras para prevenir overfitting
    'colsample_bytree': [0.7, 0.9, 1.0]  # Probar con menos columnas para hacer el modelo más robusto
}


In [24]:
# Utilizar GridSearchCV para buscar los mejores hiperparámetros
grid_search = GridSearchCV(xgb_model, param_grid_alt, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=0,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_ra...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'colsample_bytree': [0.7, 0.9, 1.0],
                         'learning_rate': [0.05, 0.15, 0.25],
                         'max_depth': [4, 6, 8], 'n_estimators': [50, 150, 250],
                         'subsample': [0.7, 0.85, 1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [35]:
best_xgb_model = grid_search.best_estimator_
print(f"Mejores hiperparámetros: {grid_search.best_params_}")

Mejores hiperparámetros: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.7}


In [37]:
# Evaluación con el mejor modelo
y_pred = best_xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

In [39]:
print(f'Error cuadrático medio (MSE) después de ajustar hiperparámetros: {mse}')
print(f'Raíz del error cuadrático medio (RMSE) después de ajustar hiperparámetros: {rmse}')

Error cuadrático medio (MSE) después de ajustar hiperparámetros: 0.003985595023881379
Raíz del error cuadrático medio (RMSE) después de ajustar hiperparámetros: 0.06313156915427795


In [41]:
# Validación cruzada con el mejor modelo
cv_scores = cross_val_score(best_xgb_model, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)
print(f'RMSE medio en validación cruzada: {cv_rmse_scores.mean()}')

RMSE medio en validación cruzada: 0.07987827014684747


In [84]:
casa_nueva = pd.DataFrame({
    'characteristics_bedrooms': [3],
    'characteristics_bathrooms': [2],
    'characteristics_garages': [2],
    'characteristics_area': [170],
    'location_lat': [-17.7675387],
    'location_lng': [-63.11527],
    'extras_Terreno': [480.0],
    'price_per_m2': [477.201708591]
})


In [66]:
casa_nueva = pd.DataFrame({
    'characteristics_bedrooms': [3],
    'characteristics_bathrooms': [2],
    'characteristics_garages': [1],
    'characteristics_area': [200.0],
    'location_lat': [-17.7922602],
    'location_lng': [-63.1949559],
    'extras_Terreno': [184.0],
    'price_per_m2': [982.3555166191]
})

In [71]:


prediccion_precio = best_xgb_model.predict(casa_nueva)

# Revertir la transformación logarítmica
precio_original = np.exp(prediccion_precio)
print(f'El precio original predicho para la casa es: ${precio_original[0]:.2f}')

El precio original predicho para la casa es: $212702.48


In [117]:
casa_nueva = pd.DataFrame({
    'characteristics_bedrooms': [4],
    'characteristics_bathrooms': [3],
    'characteristics_garages': [2],
    'characteristics_area': [254],
    'location_lat': [-17.765837436946995],
    'location_lng': [-63.201217388440114],
    'extras_Terreno': [159],
    'price_per_m2': [1006.548176793]
})


In [33]:
casa_nueva = pd.DataFrame({
    'characteristics_bedrooms': [3],
    'characteristics_bathrooms': [1],
    'characteristics_garages': [2],
    'characteristics_area': [110],
    'location_lat': [-17.768624602981113],
    'location_lng': [-63.198995590646525],
    'extras_Terreno': [273.0],
    'price_per_m2': [1006.548176793]
})


In [75]:
best_xgb_model.save_model('modelo_price_per_m2.json')